<a href="https://colab.research.google.com/github/MrCamilo5/brewingdatacup2020_kstestcol/blob/main/KSTest_col.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datos
Rutas

1.   Input1_clientes_estructura.csv
2.   Input2_clientes_venta.csv
3.  Input3_clientes_test.csv



---



1. Marca1: Marca_20 - Cupo_3 - CapacidadEnvase_9
2. Marca2: Marca_16- Cupo_2 - CapacidadEnvase_10
3. Marca3: Marca_9-Cupo_3-CapacidadEnvase_12
4. Marca_Inno1: Marca_38- Cupo_2 -CapacidadEnvase_10
5. Marca_Inno2: Marca_39-Cupo_2-CapacidadEnvase_10

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df2=pd.read_csv('Input2_clientes_venta.csv', sep=';')
df2.head()

In [ ]:
df2=df2.iloc[:,[2,4,5,6]]

**Limpieza**

In [ ]:
fdb=pd.get_dummies(df2)

In [ ]:
colacep=['Cliente',
         'Marca2_Marca_20','Marca2_Marca_16','Marca2_Marca_9','Marca2_Marca_38','Marca2_Marca_39',
         'Cupo2_Cupo_3','Cupo2_Cupo_2',
         'CapacidadEnvase2_CapacidadEnvase_9','CapacidadEnvase2_CapacidadEnvase_10','CapacidadEnvase2_CapacidadEnvase_12']

In [ ]:
allcol=fdb.columns.values
for i in allcol:
  if i in colacep:
    pass
  else:
    fdb.drop([i], axis = 'columns', inplace=True)
fdb=fdb.groupby('Cliente', sort=False).sum().astype(int)

In [ ]:
ndf=fdb.assign(Marca1=fdb.Marca2_Marca_20+fdb.Cupo2_Cupo_3+fdb.CapacidadEnvase2_CapacidadEnvase_9,
               Marca2=fdb.Marca2_Marca_16+fdb.Cupo2_Cupo_2+fdb.CapacidadEnvase2_CapacidadEnvase_10,
               Marca3=fdb.Marca2_Marca_9+fdb.Cupo2_Cupo_3+fdb.CapacidadEnvase2_CapacidadEnvase_12,
               Marca_Inno1=fdb.Marca2_Marca_38+fdb.Cupo2_Cupo_2+fdb.CapacidadEnvase2_CapacidadEnvase_10,
               Marca_Inno2=fdb.Marca2_Marca_39+fdb.Cupo2_Cupo_2+fdb.CapacidadEnvase2_CapacidadEnvase_10)

In [ ]:
allcol2=ndf.columns.values
for i in allcol2:
  if i in colacep:
    ndf.drop([i], axis = 'columns', inplace=True)
  else:
    pass
ndf.to_csv('datosres.csv')

*Escoger la muestra de los datos*

In [ ]:
resp=pd.read_csv('Input3_clientes_test.csv', sep=';')

In [ ]:
datos=pd.read_csv('datosres.csv', index_col=[0])

In [ ]:
size=(resp.shape[0],resp.shape[1]-1)
xy=np.zeros(size).astype(int)
res= resp['Cliente'].values

In [ ]:
j=0
for i in res:
  l=datos.index.get_loc(i)
  xy[j]=datos.values[l]
  j+=1

#xy --> muestra

*Prueba de bondad y ajuste*

In [ ]:
import scipy.stats as ss
from scipy.stats import kstest

In [ ]:
for i in ndf.columns.values:
  mu, s = ss.norm.fit(ndf[i])
  print(kstest(ndf[i],"norm" ,args=(mu,s)))

# Cálculo de probabilidades 

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
standar=StandardScaler()

In [ ]:
x=ndf.values
xst=standar.fit_transform(x)
x_ms=standar.fit_transform(xy)

In [ ]:
mu, st= ss.norm.fit(xst)
resultado = ss.norm(mu,st).cdf(x_ms)

In [ ]:
col=resp.columns.values[1:]

In [ ]:
j=0
for i in col:
  resp[i]=resultado[:,j]

In [ ]:
resp=resp.round(6)
resp.head()

**Resultado**

In [ ]:
resp.to_csv('Resultados_kstest.csv', sep=',', index=False)